In [89]:
import torch
from torch.cuda import Event
import time
import pandas as pd

In [90]:
print(torch.cuda.is_available())

True


## Задание 3: Сравнение производительности CPU vs CUDA

In [ ]:
# В зависимости от входных данных, выполняет соответствующую операцию с 3 тензорами
def calculate(op: str, a: torch.Tensor, b: torch.Tensor, c: torch.Tensor):
    if op == 'mul':
        return a @ a, b @ b, c @ c
    elif op == 'add':
        return a + a, b + b, c + c
    elif op == 'emul':
        return a * a, b * b, c * c
    elif op == 'transpose':
        return a.T, b.T, c.T
    elif op == 'sum':
        return a.sum(), b.sum(), c.sum()


# Функция подсчёта времени выполнения конкретной операции
# на вход должны подаваться устройство (cuda или cpu) и операция (умножение, сложение и тд)
def cpu_vs_gpu(device: str, op: str):
    operation = ['mul', 'add', 'emul', 'transpose', 'sum']

    # Валидация входных данных
    if device not in ['cuda', 'cpu']:
        raise Exception('device can be cuda or cpu')


    if device == 'cuda' and not torch.cuda.is_available():
        raise Exception('cuda is not available')


    if op not in operation:
        raise Exception("operation can be 'mul', 'add', 'emul', 'transpose', 'sum'")


    # Инициализация матриц
    a = torch.randn(64, 1024, 1024, device=device)
    b = torch.randn(128, 512, 512, device=device)
    c = torch.randn(256, 256, 256, device=device)


    # Разогрев
    calculate(op, a, b, c)


    if device == 'cuda':
        torch.cuda.synchronize()
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record() # Старт времени
        result = calculate(op, a, b, c) # Выполнение вычислений
        end_event.record() # Окончание времени
        torch.cuda.synchronize()
        elapsed_time = start_event.elapsed_time(end_event) / 1000  # В секундах
        return f'{elapsed_time:.4f}'
    else:
        start_time = time.time() # Старт времени
        result = calculate(op, a, b, c) # Выполнение вычислений
        elapsed_time = time.time() - start_time # Подсчёт времени (в сек.)
        return f'{elapsed_time:.4f}'

Функция измерения времени

In [ ]:
# CUDA
cuda_test_mul = cpu_vs_gpu('cuda', 'mul')
cuda_test_add = cpu_vs_gpu('cuda', 'add')
cuda_test_emul = cpu_vs_gpu('cuda', 'emul')
cuda_test_transpose = cpu_vs_gpu('cuda', 'transpose')
cuda_test_sum = cpu_vs_gpu('cuda', 'sum')

# CPU
cpu_test_mul = cpu_vs_gpu('cpu', 'mul')
cpu_test_add = cpu_vs_gpu('cpu', 'add')
cpu_test_emul = cpu_vs_gpu('cpu', 'emul')
cpu_test_transpose = cpu_vs_gpu('cpu', 'transpose')
cpu_test_sum = cpu_vs_gpu('cpu', 'sum')

In [93]:
data = {
    'Операция': ['Матричное умножение', 'Поэлементное сложение', 'Поэлементное умножение', 'Транспонирование', 'Сумма'],
    'CUDA (сек)': [cuda_test_mul, cuda_test_add, cuda_test_emul, cuda_test_transpose, cuda_test_sum],
    'CPU (сек)': [cpu_test_mul, cpu_test_add, cpu_test_emul, cpu_test_transpose, cpu_test_sum]
}


df = pd.DataFrame(data)
df

,Операция,CUDA (сек),CPU (сек)
0,Матричное умножение,0.0626,2.0314
1,Поэлементное сложение,0.0039,0.2349
2,Поэлементное умножение,0.0039,0.2279
3,Транспонирование,0.0000,0.0000
4,Сумма,0.0018,0.0385


___
### Выводы

_Я делал в среде Google Colab, так как моё устройство не поддерживает CUDA._

Вычисление на графическом процессоре дает очень большой прирост в скорости, что видно в результативной таблице. В среднем операции на GPU выполняются в 40 раз быстрее - разница колосальна. Объясняется это тем, что GPU изначально создан для вычисления большого количество информации **параллельно**, когда процессоры в свою очередь созданы для более сложных вычислений в целом, но с меньшим количеством потоков. 

Самой долгой операцией является матричное умножение, вероятно, потому что операция сама по себе является очень тяжелой и имеет сложность O(n^3).

Также можно увидеть, что транспонирование выполнилось моментально (0 сек.). Я не нашёл информации о реализации транспонирования в NumPy, но нейронная сеть ответила, что для данной операции сложность O(1), так как всё сводится к перестановке указателей, плюс при распараллеливании время может быть ничтожно маленьким, потому результат 0.

Подводя итоги, работа с GPU даёт большой прирост к вычислениям, особенно это будет заметно на ещё большем количестве данных.